In [2]:
import torch 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class MyMLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(5,11),
            torch.nn.ReLU(),
            torch.nn.Linear(11,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,13),
            torch.nn.ReLU(),
            torch.nn.Linear(13,8),
            torch.nn.ReLU(),
            torch.nn.Linear(8,4),
            torch.nn.Softmax(dim=1)
        )
    def forward(self,x):
        return self.layers.forward(x)


In order to calculate the exact number of parameters for each layer suppose to have the number of neurons in each layer:
$d_1, d_2,\cdots , d_n$

This means that each neuron in the $k^{th}$ layer is connected to $d_{k+1}$ neurons. So the total number of parameters (bias excluded) 
needed in the $k^{th}$ layer is $d_k * d_{k+1}$

So overall the total number of parameters is 
$$|W| = \sum_{i = 1} ^ {n-1} d_i * d_{i+1}$$

If we consider also the bias as an additional weight the formula becomes 
$$|W| = \sum_{i = 1} ^ {n-1} (d_i + 1) * d_{i+1}$$

In [4]:
mlp = MyMLP()

import numpy as np
#looping through parameters
#actually the method parameters returns biases and weights as different tensors
#for matrices `torch.linalg.norm` returns the Frobenious norm, for vector the euclidean norm is returned
for p in mlp.parameters():
    print(p.shape, '\t',f"{torch.linalg.norm(p)} ")

from torchinfo import summary
summary(mlp)

torch.Size([11, 5]) 	 1.857778787612915 	 
torch.Size([11]) 	 0.8217586278915405 	 
torch.Size([16, 11]) 	 2.3974788188934326 	 
torch.Size([16]) 	 0.8743267059326172 	 
torch.Size([13, 16]) 	 2.1002204418182373 	 
torch.Size([13]) 	 0.5187804698944092 	 
torch.Size([8, 13]) 	 1.644871473312378 	 
torch.Size([8]) 	 0.40382179617881775 	 
torch.Size([4, 8]) 	 1.065483570098877 	 
torch.Size([4]) 	 0.5143529772758484 	 


Layer (type:depth-idx)                   Param #
MyMLP                                    --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       66
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       192
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       221
│    └─ReLU: 2-6                         --
│    └─Linear: 2-7                       112
│    └─ReLU: 2-8                         --
│    └─Linear: 2-9                       36
│    └─Softmax: 2-10                     --
Total params: 627
Trainable params: 627
Non-trainable params: 0

In [5]:
#generate 10 random datapoints
x = torch.rand(10,5)

In [6]:
#forward pass through the network
y = mlp.forward(x)
print(y)

tensor([[0.3138, 0.1916, 0.2080, 0.2867],
        [0.3160, 0.1893, 0.2052, 0.2895],
        [0.3158, 0.1902, 0.2067, 0.2872],
        [0.3140, 0.1905, 0.2078, 0.2877],
        [0.3145, 0.1918, 0.2082, 0.2855],
        [0.3152, 0.1908, 0.2078, 0.2862],
        [0.3141, 0.1922, 0.2081, 0.2855],
        [0.3156, 0.1907, 0.2062, 0.2875],
        [0.3143, 0.1909, 0.2074, 0.2875],
        [0.3154, 0.1906, 0.2076, 0.2864]], grad_fn=<SoftmaxBackward0>)


In [10]:
#argmax returns the index for each row of the largest value
#we are supposing that the example is classified according to 

y_class = torch.argmax(y, axis = 1)
print(f"Neural network classification: {y_class}")
y_true = torch.randint_like(y_class,0,4)
print(f"Random integer tensor to test the accuracy score against: {y_true}")

Neural network classification: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Random integer tensor to test the accuracy score against: tensor([0, 0, 3, 2, 2, 0, 3, 0, 2, 0])


In [11]:
def accuracy(y_pred,y_true):
    y_eq = (y_pred == y_true)
    y_eq.int()
    return float(torch.sum(y_eq)/y_eq.shape[0])

In [12]:
accuracy(y_class,y_true)

0.5